In [ ]:
import azureml.core

print(azureml.core.VERSION)

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(".azureml")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"
vm_size = "Standard_D2_V2"
max_nodes = 4

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster. use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=max_nodes)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# a detailed status for the current cluster.
print(compute_target.get_status().serialize())

In [ ]:
import os
import shutil

from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig

# Specify parameter sampler
parameter_space = { "--C": uniform(0.1, 1.0), 
                    "--max_iter": choice([10, 25, 50, 100, 500]) }
param_sampling = RandomParameterSampling(parameter_space) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=5) ### YOUR CODE HERE ###

script_folder = "training"
os.makedirs(os.path.join(".", script_folder), exist_ok=True)

# Create a SKLearn estimator for use with train.py
shutil.copy("./train.py", os.path.join(".", script_folder))
source_directory=os.path.join(".", script_folder)



In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-defaults

In [ ]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [ ]:
# Use ScrptRunConfig
src = ScriptRunConfig(source_directory=source_directory,
                      script='train.py',
                      arguments=[],
                      compute_target=compute_target,
                      environment=sklearn_env)

In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                        run_config=src,
                        hyperparameter_sampling=param_sampling,
                        policy=policy,
                        primary_metric_name="accuracy",
                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                        max_total_runs=25,
                        max_concurrent_runs=max_nodes-1
                        ) 

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)

RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()["runDefinition"]["arguments"]


In [ ]:
print("Best Run Id      :", best_run.id)
print("Accuracy         :", best_run_metrics["accuracy"])
print("--C              :", parameter_values[1])
print("--max-iter       :", parameter_values[3])

print(parameter_values)

In [ ]:
best_run_filenames = best_run.get_file_names()
for filename in best_run_filenames:
    print(filename)

In [ ]:
# the last element of best_run_filenames is a joblib file.
best_run.download_file(best_run_filenames[-1], output_file_path=os.path.join(".", script_folder))

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
filename = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(filename)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds) 

x["y"] = y

os.makedirs(os.path.join(".", "data"), exist_ok=True)
x.to_csv(os.path.join(".", "data", "bankmarketing_prepared.csv"))

datastore = ws.get_default_datastore()
datastore.upload(os.path.join(".", "data"), target_path="data")

ds_prepared = TabularDatasetFactory.from_delimited_files(datastore.path("data/bankmarketing_prepared.csv"))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    task="classification",
    experiment_timeout_minutes=30,
    primary_metric="accuracy",
    compute_target=compute_target,
    training_data=ds_prepared,
    label_column_name="y",
    n_cross_validations=5,
    max_cores_per_iteration=-1,
    max_concurrent_iterations=max_nodes,
    enable_onnx_compatible_models=True
    )

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)

RunDetails(automl_run).show()

In [ ]:
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, best_model = automl_run.get_output()

best_model

In [ ]:
# remove compute target
compute_target.delete()